In [ ]:
import numpy as np
import tifffile as tiff
from pathlib import Path
import tifffile as tiff
from scipy.ndimage import label, center_of_mass
import os

# windows is the antichrist
cwd = os.getcwd()
print(cwd)

# load tif mask as ndarray
nuc = tiff.imread("nucmask_1.tiff")
print(nuc.shape)

# sample centroids
labeled_mask, num_labels = label(nuc)

# Calculate the centroid for each nucleus (label)
centroids = center_of_mass(nuc, labeled_mask, range(1, num_labels + 1))
point_cloud = np.array(centroids)  # Convert to array format for further processing
pc3d = point_cloud


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance

#

# plot distribution of pairwise distances
pairwise_distances = distance.pdist(point_cloud, metric='euclidean')
plt.hist(pairwise_distances, bins=30, color='blue', alpha=0.7)
plt.title("Distribution of Pairwise Distances")
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.show()

# Step 3: Compute statistics for guidance (e.g., percentiles)
min_distance = np.min(pairwise_distances)
max_distance = np.max(pairwise_distances)
mean_distance = np.mean(pairwise_distances)
median_distance = np.median(pairwise_distances)
percentile_90 = np.percentile(pairwise_distances, 90)

print(f"Min distance: {min_distance}")
print(f"Max distance: {max_distance}")
print(f"Mean distance: {mean_distance}")
print(f"Median distance: {median_distance}")
print(f"90th percentile: {percentile_90}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import label, center_of_mass

shape = nuc.shape
middle_index = shape[0] // 2  
middle_slice = nuc[middle_index, :, :]

# Visualize to verify (optional)
import matplotlib.pyplot as plt
plt.imshow(middle_slice, cmap='gray')
plt.title(f"Middle Slice (index {middle_index})")
plt.show()

###
# Label connected components (nuclei) in the middle slice
labeled_slice, num_features = label(middle_slice)

# Calculate centroids of the connected components
centroids = center_of_mass(middle_slice, labeled_slice, range(1, num_features + 1))

# Print or store the centroids for further use
print(f"Extracted {len(centroids)} centroids from the slice.")
print("Centroids:", centroids)

# Convert centroids to a NumPy array for further processing if needed
centroid_points = np.array(centroids)

# Plot the middle slice
plt.imshow(middle_slice, cmap='gray')

# Plot the centroids as red dots
if len(centroid_points) > 0:
    plt.scatter([c[1] for c in centroid_points], [c[0] for c in centroid_points], color='red',  s=10)

plt.title("Nuclear Centroids on Middle Slice")
plt.show()

In [ ]:
import gudhi as gd
import numpy as np
import matplotlib.pyplot as plt

## define point cloud:
point_cloud = centroid_points
#point_cloud = pc3d

# Filtration range and step size
max_edge_length = max_distance
step = 0.1
filtration_values = np.arange(0, max_edge_length, step)

# Create a Rips complex and simplex tree
rips_complex = gd.RipsComplex(points=point_cloud, max_edge_length=max_edge_length)
simplex_tree = rips_complex.create_simplex_tree(max_dimension=2)

# Store all simplices from the filtration
all_simplices = list(simplex_tree.get_filtration())

# Function to calculate Euler characteristics for a range of filtration values
def compute_euler_characteristics(filtration_values):
    euler_values = []
    for f_value in filtration_values:
        V = sum(1 for simplex, filtration in all_simplices if len(simplex) == 1 and filtration <= f_value)
        E = sum(1 for simplex, filtration in all_simplices if len(simplex) == 2 and filtration <= f_value)
        F = sum(1 for simplex, filtration in all_simplices if len(simplex) == 3 and filtration <= f_value)
        euler_values.append(V - E + F)
    return euler_values

# Compute Euler characteristics over filtration range
euler_characteristics = compute_euler_characteristics(filtration_values)

# Calculate centered Euler characteristics by subtracting the mean
centered_euler_characteristics = euler_characteristics - np.mean(euler_characteristics)

# Perform cumulative integration for the smooth Euler characteristic
smooth_euler_characteristic = np.cumsum(centered_euler_characteristics) * step  # Multiply by step for cumulative integration

# Plot the original and smooth Euler characteristic curves
plt.plot(filtration_values, euler_characteristics, label='Euler Characteristic')
plt.plot(filtration_values, smooth_euler_characteristic, label='Smooth Euler Characteristic')
plt.xlabel('Filtration Radius (r)')
plt.ylabel('Euler Characteristic (χ)')
plt.title('Euler Characteristic Curve and Smooth Euler Characteristic')
plt.legend()
plt.show()


## Scratch


In [ ]:
# chunk mask into smaller groups to test how ec curve distinguishes them
import numpy as np

# Assuming 'segmentation_mask' is your 256x256x256 array
segmentation_mask = nuc
chunked_mask = segmentation_mask.reshape(4, 64, 4, 64, 4, 64).swapaxes(1, 2).reshape(-1, 64, 64, 64)

# Verify by printing the shape of the chunks
print("Chunked mask shape:", chunked_mask.shape)
import matplotlib.pyplot as plt
import numpy as np

# Assume 'segmentation_mask' is your 3D 256x256x256 array
# Replace this with your actual data
segmentation_mask = np.ones((256, 256, 256))  # Example data

# Define chunk size and slice positions
chunk_size = 64
mid_x, mid_y, mid_z = 128, 128, 128  # Middle points for slicing

# Plot slices with grid overlay
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# XY plane at mid_z
axs[0].imshow(segmentation_mask[:, :, mid_z], cmap='gray')
axs[0].set_title('XY plane at Z=128')
for i in range(0, 256, chunk_size):
    axs[0].axhline(i, color='red', linestyle='--', linewidth=0.5)  # Horizontal grid lines
    axs[0].axvline(i, color='red', linestyle='--', linewidth=0.5)  # Vertical grid lines

# XZ plane at mid_y
axs[1].imshow(segmentation_mask[:, mid_y, :], cmap='gray')
axs[1].set_title('XZ plane at Y=128')
for i in range(0, 256, chunk_size):
    axs[1].axhline(i, color='blue', linestyle='--', linewidth=0.5)  # Horizontal grid lines
    axs[1].axvline(i, color='blue', linestyle='--', linewidth=0.5)  # Vertical grid lines

# YZ plane at mid_x
axs[2].imshow(segmentation_mask[mid_x, :, :], cmap='gray')
axs[2].set_title('YZ plane at X=128')
for i in range(0, 256, chunk_size):
    axs[2].axhline(i, color='green', linestyle='--', linewidth=0.5)  # Horizontal grid lines
    axs[2].axvline(i, color='green', linestyle='--', linewidth=0.5)  # Vertical grid lines

# Display the plot
plt.tight_layout()
plt.show()

